In [5]:
%load_ext autoreload
%autoreload 2

import smtplib as smtp 
import ssl 
import os

import queue
import multiprocessing

from functions import *
from master_functions import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Mail

In [2]:
mail = 'drainingytb@gmail.com'
pswd = 'YoDH@ck3r'

In [3]:
ssl_cntxt = ssl.create_default_context()

In [14]:
smtp.SMTP_SSL()


with smtplib.SMTP_SSL("smtp.gmail.com", port=465, context=ssl_cntxt) as server:
    server.login(mail, pswd)
    server.sendmail(from_addr=mail,
                    to_addrs='xabier.rubiato@epfl.ch',
                    msg='Finish Process')

## Download

In [3]:
from functions import yt_dl

### Vine playlist test 

In [2]:
URL = 'https://www.youtube.com/watch?v=2S7Yak00e6E&list=PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b'
PLAYLIST = 'test'
N_ITEMS = 6 
PTH_PLYLST = pth_plylst(name=PLAYLIST)

In [4]:
print('Start download of playlist : {}\n'.format(PLAYLIST))
yt_dl(url= URL,
      playlist=PLAYLIST,
      n_items= N_ITEMS)
print('Finish download of playlist : {}\n'.format(PLAYLIST))

Start download of playlist : test

[youtube:playlist] Downloading playlist PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b - add --no-playlist to just download video 2S7Yak00e6E
[youtube:playlist] PLeu1bSfyWCDK4KgsF2pJybJkWXKPHgD6b: Downloading webpage
[download] Downloading playlist: Alignments as vines
[youtube:playlist] playlist Alignments as vines: Downloading 6 videos
[download] Downloading video 1 of 6
[youtube] 2S7Yak00e6E: Downloading webpage
[youtube] 2S7Yak00e6E: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/2S7Yak00e6E/data/2S7Yak00e6E_1920x1080.info.json


[download] videos/test/2S7Yak00e6E/data/2S7Yak00e6E_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 2 of 6
[youtube] qoqZ9T5O29g: Downloading webpage
[youtube] qoqZ9T5O29g: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/qoqZ9T5O29g/data/qoqZ9T5O29g_1920x1080.info.json


[download] videos/test/qoqZ9T5O29g/data/qoqZ9T5O29g_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 3 of 6
[youtube] IMVmevu6mE4: Downloading webpage
[youtube] IMVmevu6mE4: Downloading embed webpage
[youtube] IMVmevu6mE4: Refetching age-gated info webpage
[info] Writing video description metadata as JSON to: videos/test/IMVmevu6mE4/data/IMVmevu6mE4_1920x1080.info.json
[download] videos/test/IMVmevu6mE4/data/IMVmevu6mE4_1920x1080.mp4 has already been downloaded and merged
[download] Downloading video 4 of 6
[youtube] MNMYYEdJoC0: Downloading webpage
[youtube] MNMYYEdJoC0: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/MNMYYEdJoC0/data/MNMYYEdJoC0_1920x1080.info.json
[download] videos/test/MNMYYEdJoC0/data/MNMYYEdJoC0_1920x1080.mp4 has already been downloaded and merged
[download] Downloading video 5 of 6
[youtube] -Ia18bqO-9w: Downloading webpage
[youtube] -Ia18bqO-9w: Downloading video info webpage
[inf

[download] videos/test/-Ia18bqO-9w/data/-Ia18bqO-9w_1920x1080.mkv has already been downloaded and merged
[download] Downloading video 6 of 6
[youtube] yd7d51AgSso: Downloading webpage
[youtube] yd7d51AgSso: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.info.json
[download] Destination: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f248.webm
[download] 100% of 43.28MiB in 00:1157MiB/s ETA 00:005
[download] Destination: videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f251.webm
[download] 100% of 4.08MiB in 00:0168MiB/s ETA 00:004
[ffmpeg] Merging formats into "videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.webm"
Deleting original file videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f248.webm (pass -k to keep)
Deleting original file videos/test/yd7d51AgSso/data/yd7d51AgSso_1920x1080.f251.webm (pass -k to keep)
[download] Finished downloading playlist: Alignments as vines
Finish downloa

### Download one video and put it into playlist 

In [4]:
URL = 'https://www.youtube.com/watch?v=PqyPW-Bdd4E'
PLYLST = 'test2'
yt_dl(URL, PLYLST)

[youtube] PqyPW-Bdd4E: Downloading webpage
[youtube] PqyPW-Bdd4E: Downloading video info webpage
[info] Writing video description metadata as JSON to: videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.info.json


[download] Destination: videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.f299.mp4
[download] 100% of 222.50MiB in 00:2081MiB/s ETA 00:001nown ETA
[download] Destination: videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.f251.webm
[download] 100% of 6.20MiB in 00:0064MiB/s ETA 00:0006
[ffmpeg] Merging formats into "videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.mkv"
Deleting original file videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.f299.mp4 (pass -k to keep)
Deleting original file videos/test2/PqyPW-Bdd4E/data/PqyPW-Bdd4E_1920x1080.f251.webm (pass -k to keep)


## Multiprocessing

### Worker class and functions

In [24]:
%load_ext autoreload
%autoreload 2

import os
from functions import *
from functions import pth_plylst
from master_functions import iter0, sfm_pipe
import multiprocessing 
import queue 
from multiprocessing import Process, Queue, cpu_count

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
class Worker: 
    
    def __init__(self, task_queue, name):
        self.q = task_queue
        self.name = name 
        
    def Qiter0(self,pth_vid):
        """Performs iter0() and enqueues an sfm_pipe process with the generated sets."""
        pths = iter0(path_vid=pth_vid)
        pth_sets = [os.path.join(pth, el) for el in os.listdir(pths)]

        # Enqueue next tasks
        for pth_set in pth_sets:  
            q.put(sfm_pipe, pth_set)
    
    def sfm_pipe(self, pth):
        print('execution of sfm_pipe')
        pass

In [84]:
def worker_exec(task_queue, name):
    # Work until no tasks are pending 
    while not task_queue.empty():
        # instanciate worker 
        worker = Worker(task_queue, name)
        task = task_queue.get()
        
        if task[0] == 'Qiter0':
            worker.Qiter0(task[1])
            
        elif task[0] == 'sfm_pipe':
            worker.sfm_pipe(task[1])

### Launch processes: 

In [86]:
NUMBER_PROCESSES = 2 #cpu_count()
PLYLST = 'test2'
path_plylst = pth_plylst(PLYLST)

# Create Queue 
Q = Queue()

# Fille queue with the iter0 jobs 
for v_id in os.listdir(path_plylst): 
    pth = pth_vid(v_id, PLYLST)
    Q.put(('Qiter0', pth))

for i in range(NUMBER_PROCESSES):
    p = Process( target=worker_exec, args=(Q,'w{}'.format(i)) )
    p.start()
    p.join()

put in queue :/Users/xrubiato/DHLab/Draining-Youtube/videos/test2/PqyPW-Bdd4E


Process Process-85:
Traceback (most recent call last):
  File "/Users/xrubiato/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/xrubiato/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-84-051e5e5e7a82>", line 9, in worker_exec
    worker.Qiter0(task[1])
  File "<ipython-input-83-59967cb5b953>", line 9, in Qiter0
    pths = iter0(path_vid=pth_vid)
  File "/Users/xrubiato/DHLab/Draining-Youtube/master_functions.py", line 62, in iter0
    pth_sets =  make_sets(v_id, plylst)
  File "/Users/xrubiato/DHLab/Draining-Youtube/master_functions.py", line 28, in make_sets
    path_frames=pth_frms(v_id, plylst)))
  File "/Users/xrubiato/DHLab/Draining-Youtube/functions.py", line 461, in split_triangles
    while j != length and abs(non_zer[j+1] - non_zer[j]) < tol:
IndexError: index 23 is out of bounds for axis 0 with size 23


## Tests

In [87]:
from master_functions import make_sets

In [88]:
path_plylst

'/Users/xrubiato/DHLab/Draining-Youtube/videos/test2'

In [90]:
v_id = os.listdir(path_plylst)[0]

In [91]:
make_sets(v_id, PLYLST)

IndexError: index 23 is out of bounds for axis 0 with size 23